In [5]:
from Polygon import Vertex, Polygon, initialize_a_polygon
from FIREminimization import calculateForceVelocityProjections, FIREminimize
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch

def plot_configuration(polygon, filename = "initial.png"):
    fig, ax = plt.subplots(figsize=(10,10))
    for i in range(len(polygon.vertices_)):
        # ax.plot(P.vertices_[i].position_,P.vertices_[i-1].position_,color='black')
        ax.plot(polygon.vertices_[i].position_[0],polygon.vertices_[i].position_[1],'o',color='black')
        # Only draw the force if it is not if it is not too small
        if np.linalg.norm(polygon.vertices_[i].force_) > 1e-15:
            ForceArrow = FancyArrowPatch(posA = polygon.vertices_[i].position_,
                                    posB = np.add(polygon.vertices_[i].position_,polygon.vertices_[i].force_),
                                    arrowstyle='->,head_width=5,head_length=10',
                                    color='blue', linewidth=2)
            ax.add_patch(ForceArrow)
        EdgeArrow = FancyArrowPatch(posA = polygon.vertices_[i].position_,
                                posB = polygon.vertices_[i-1].position_,
                                arrowstyle='-',
                                color='black', linewidth=2)

        ax.add_patch(EdgeArrow)
        


        # ax.add_patch(arrow)
    plt.xlim(-2, 2)
    plt.ylim(-2, 2)
    plt.savefig(filename)
    plt.close()

    return

In [6]:
P = initialize_a_polygon(P0 = 3.77)


print("Initial area: ", P.area_)
print("Initial perimeter: ", P.perimeter_)
print("initial stress:\n", P.stress_)
plot_configuration(P, "initial.png")
FIREminimize(P)

P.compute_forces()
P.compute_stress()
plot_configuration(P, "final.png")
print("Minimized perimeter ",P.perimeter_)
print("Minimized area ",P.area_)
print("Minimized stress:\n",P.stress_)
print("Forces after minimization:")
for i in range(len(P.vertices_)):
    print(P.vertices_[i].force_)


Initial area:  0.9696860125418824
Initial perimeter:  3.65438547986877
initial stress:
 [[0.52829199 0.01529391]
 [0.01529391 0.48210903]]
Iteration  0 ff =  0.3729049147367618 vv =  3.7290491473676185e-07 power =  0.00037290491473676183
Minimized perimeter  3.77
Minimized area  1.0
Minimized stress:
 [[0. 0.]
 [0. 0.]]
Forces after minimization:
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
